In [1]:
import numpy as np
import pandas as pd

In [2]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [3]:
def compare_fuzzy(desc,buyer_name):
    return fuzz.partial_ratio(desc,buyer_name)

In [4]:
bank = pd.read_csv('../input/payment-id-ndsc-2020/bank_statement.csv')
checkout = pd.read_csv('../input/payment-id-ndsc-2020/checkout.csv')

In [5]:
bank.head()

,stmt_id,stmt_amount,desc
0,1,100.00,INSTANT TRANSFER BAW CGANBAWI YIWHI * LEZC ...
1,2,23.77,WOOG CARIZA NIWSI YO
2,3,18.00,ZOMAZAW MABIX ATMA * BUGW * KZBUGW - TRANSFER...
3,4,21.87,TRANSFER GO LA/FA AHHA PUYIMA * YEBF ...
4,5,53.68,INSTANT TRANSFER FABZUN WIQAT JIW FA* TEGF ...


In [6]:
checkout.head()

,ckt_id,ckt_amount,buyer_name
0,1,20.00,SAVMUEJICE EPHOCP
1,2,14.91,LUHALLAK ZGANFAP HAIQAW VIP LOHK. CIQTI
2,3,13.20,SUQ UNAISIE
3,4,36.90,GOZK BIHKAUT LIV LATAPI
4,7,26.29,HUY ZABIRAN HATIZAN WIHMI IRYIZ


In [7]:
len(checkout)

240000

In [8]:
len(bank)

240000

In [9]:
def find_match(stmt_amount, desc):
    potential_match = checkout[checkout['ckt_amount']==stmt_amount]
    result = []
    for ckt_id, buyer_name in zip(potential_match['ckt_id'], potential_match['buyer_name']):
        score = compare_fuzzy(desc, buyer_name)
        if score > 0:
            result.append((ckt_id,score))
    return result

In [10]:
def get_max(list_of_tuple):
    max_score = 0
    final_id = -1
    for stmt_id, score in list_of_tuple:
        if score > max_score:
            max_score = score
            final_id = stmt_id
    return final_id

In [11]:
final_result = []

In [12]:
for stmt_id, stmt_amount, desc in zip(bank['stmt_id'], bank['stmt_amount'], bank['desc']):
    ckt_id = get_max(find_match(stmt_amount, desc))
    final_result.append((stmt_id,ckt_id))

In [13]:
stmt_id = [a for a,b in final_result]
ckt_id = [b for a,b in final_result]
df = pd.DataFrame({'stmt_id':stmt_id, 'ckt_id': ckt_id})
df.head()

,stmt_id,ckt_id
0,1,37064
1,2,37067
2,3,214638
3,4,137482
4,5,95564


In [14]:
df.to_csv('hasil_akhir.csv', index=False)